In [1]:
import importlib, os, gsw, gc

import pandas as pd
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
%matplotlib widget
import matplotlib.dates as mdates

import SteadyState2Constraints as ss2
import SXBQ as sx
importlib.reload(ss2)
importlib.reload(sx)

<module 'SXBQ' from 'C:\\Users\\johan\\OneDrive\\Bureau\\PFE-Goteborg\\Scripts\\SXBQ.py'>

# LOAD HYDROGRAPHIC DATA

In [2]:
data = sx.sxdf()
data.data = pd.read_parquet('C://Users/johan/OneDrive/Bureau/PFE-Goteborg/results.pqt')

top_mounted = True

def delimitateProfiles():
    #data.data.sort_values('Timestamp', ignore_index=True, inplace=True)
    data.median_resample()

    _gd = np.isfinite(data.data['diveNum'].values)
    _, _tmp = np.unique( np.round(data.data['diveNum'].values[_gd]) , return_inverse=True)

    data.data.loc[_gd,'diveNum'] = np.round(_tmp+1)
    data.data['diveNum'] = data.data['diveNum'].interpolate('nearest')
        
    data.data['profileNum'] = data.data['diveNum'].values*2
    _tmp = data.data['NAV_RESOURCE'].interpolate('nearest').values
    ind = (_tmp == 100) | (_tmp == 110) | (_tmp == 116)
    data.data.loc[ind,'profileNum'] = data.data.loc[ind,'profileNum'] - 1

delimitateProfiles()

def rmsd(x):
    return np.sqrt(np.nanmean(x**2))

#data.data

# LOAD ADCP DATA

In [3]:
#for first set of data
adcp_path = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA045_M33/ADCP/sea045_M33_A0'

header = ['Month','Day','Year','Hour','Minute','Second','Burst counter','Ensemble counter','Error code','Status code','Battery voltage','Soundspeed','Heading','Pitch','Roll','Pressure','Temperature','Analog input 1','Analog input 2']
ADCP = pd.read_table(adcp_path+'.sen',sep=r"\s+",names=header)
ADCP.insert(0,'time',pd.to_datetime(ADCP[['Year', 'Month', 'Day', 'Hour', 'Minute','Second']], utc=True, origin='unix', cache='False'))

ADCP.set_index('time', inplace=True)

ADCP = ADCP.to_xarray()

for beam in ['1','2','3','4']:
    ADCP['V'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path+'.v'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )
    ADCP['C'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path+'.c'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )

ADCP

<xarray.Dataset>
Dimensions:           (bin: 15, time: 183468)
Coordinates:
  * time              (time) object 1597909692438000000 ... 1600074055438000000
Dimensions without coordinates: bin
Data variables: (12/27)
    Month             (time) int64 8 8 8 8 8 8 8 8 8 8 8 ... 9 9 9 9 9 9 9 9 9 9
    Day               (time) int64 20 20 20 20 20 20 20 ... 10 10 10 10 14 14 14
    Year              (time) int64 2020 2020 2020 2020 ... 2020 2020 2020 2020
    Hour              (time) int64 7 7 7 7 7 7 7 7 7 ... 14 14 14 14 14 14 9 9 9
    Minute            (time) int64 48 48 48 48 48 49 49 49 ... 7 7 7 7 7 0 0 0
    Second            (time) float64 12.44 22.44 32.44 ... 35.44 45.44 55.44
    ...                ...
    V2                (time, bin) float64 0.127 -0.083 -0.627 ... 0.971 1.979
    C2                (time, bin) float64 48.0 0.0 1.0 0.0 ... 2.0 3.0 1.0 2.0
    V3                (time, bin) float64 0.0 -1.065 -1.168 ... 1.946 0.748
    C3                (time, bin) float64 56.0 2.0 3.0 1.0 ... 0.0 2.0 5.0 2.0
    V4                (time, bin) float64 0.1 2.003 -0.928 ... -2.223 1.973
    C4                (time, bin) float64 47.0 0.0 0.0 0.0 ... 3.0 0.0 1.0 2.0

In [4]:
print(ADCP.V1.time)

<xarray.DataArray 'time' (time: 183468)>
array([1597909692438000000, 1597909702438000000, 1597909712438000000, ...,
       1600074035438000000, 1600074045438000000, 1600074055438000000],
      dtype=object)
Coordinates:
  * time     (time) object 1597909692438000000 ... 1600074055438000000


# CALCULATE ADCP BIN DEPTH

In [5]:
def getADCPBinDepth(ADCP, bin_size, blanking_distance,direction='down',lat=58):
    # Retuns a new coordinate within the ADCP matrix of size ntime x nbin containing ADCP bin depths
    if top_mounted:
        direction = 1
    else:
        direction = -1
 
    #PREVIOUS CORRECTIONS (alpha here is angle between Zadcp and Z geographic)
    #bin_distance = blanking_distance + np.arange(len(ADCP.bin))*bin_size + 0.5*bin_size
    #alpha = np.arccos(
    #            np.cos(np.deg2rad(22.5 + ADCP['Pitch'])) * # THIS IS WRONG
    #            np.cos(np.deg2rad(ADCP['Roll'])) 
    #        )
    
    bin_distance = (blanking_distance + np.arange(len(ADCP.bin))*bin_size + 0.5*bin_size) *np.cos(np.deg2rad(22.5)) 
    alpha = np.arccos(
                np.cos(np.deg2rad(22.5))*np.cos(np.deg2rad(ADCP['Pitch']))*np.cos(np.deg2rad(ADCP['Roll']))
                + np.abs(   np.sin(np.deg2rad(22.5))*np.sin(np.deg2rad(ADCP['Pitch']))   )
            )
    
    adcp_depth = gsw.z_from_p(ADCP['Pressure'],lat)  #Depth is negatice underwater

    bin_depth = np.tile(adcp_depth, (len(ADCP.bin), 1)).T \
                + direction \
                * np.tile(bin_distance, (len(ADCP.time), 1)) \
                * np.tile(np.cos(alpha), (len(ADCP.bin), 1)).T \

    return ADCP.assign_coords({'bin_depth':(['time','bin'], bin_depth)})

ADCP = getADCPBinDepth(ADCP,2,0.2,55) #Or - in direction ?

# SOUNDSPEED CORRECTION

In [6]:
# Not implemented yet

# Rotation correction

In [7]:
def rot_correction ():
    ADCP['dPitch_dt'] = (['time'], np.gradient(   -np.deg2rad(ADCP.Pitch.values.astype('float'))   ,ADCP.time.values.astype('float')/(10**9)))
    dx=0.25
    dz=0.1
    a=47.5
    b=25
    ADCP['V1'] = ADCP['V1'] - ADCP['dPitch_dt']*(-dz*np.sin(np.deg2rad(a))+dx*np.cos(np.deg2rad(a)))
    ADCP['V3'] = ADCP['V3'] - ADCP['dPitch_dt']*(dz*np.sin(np.deg2rad(a))+dx*np.cos(np.deg2rad(a)))
    ADCP['V2'] = ADCP['V2'] - ADCP['dPitch_dt']*( dx*np.cos(np.deg2rad(b)) )
    ADCP['V4'] = ADCP['V4'] - ADCP['dPitch_dt']*( dx*np.cos(np.deg2rad(b)) )

#rot_correction()

# 3 BEAM PITCH-DEPENDENT SOLUTION TO XYZ AND ENU CALCULATION

In [8]:
def calcXYZfrom3beam_bis():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam_fore = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])
    
    xyz2beam_aft = np.array([
        [-sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])

    beam2xyz_fore = np.linalg.inv(xyz2beam_fore)
    beam2xyz_aft = np.linalg.inv(xyz2beam_aft)

    V_fore = beam2xyz_fore @ np.array([
        ADCP['V1'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])
    V_aft = beam2xyz_aft @ np.array([
        ADCP['V3'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')

    X_fore = np.reshape( V_fore[0,:] , (-1,15) ) #Because number of cell is 60
    X_aft = np.reshape( V_aft[0,:] , (-1,15) ) #Because number of cell is 60
    Y_fore = np.reshape( V_fore[1,:] , (-1,15) ) #Because number of cell is 60
    Y_aft = np.reshape( V_aft[1,:] , (-1,15) ) #Because number of cell is 60
    Z_fore = np.reshape( V_fore[2,:] , (-1,15) ) #Because number of cell is 60
    Z_aft = np.reshape( V_aft[2,:] , (-1,15) ) #Because number of cell is 60
    
    
    use_aft_on_climb = ADCP['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X_fore[~use_aft_on_climb,:] = X_aft[~use_aft_on_climb,:]
        Y_fore[~use_aft_on_climb,:] = Y_aft[~use_aft_on_climb,:]
        Z_fore[~use_aft_on_climb,:] = Z_aft[~use_aft_on_climb,:]
        
    else:
        print('Assuming ADCP is bottom mounted')
        X_fore[use_aft_on_climb,:] = X_aft[use_aft_on_climb,:]
        Y_fore[use_aft_on_climb,:] = Y_aft[use_aft_on_climb,:]
        Z_fore[use_aft_on_climb,:] = Z_aft[use_aft_on_climb,:]
        Y_fore=-Y_fore
        Z_fore=-Z_fore

    ADCP['X'] = (['time','bin'], X_fore )
    ADCP['Y'] = (['time','bin'], Y_fore ) 
    ADCP['Z'] = (['time','bin'], Z_fore ) 
    
    _ = plt.hist(X_fore.flatten(),100,color='g',alpha=0.3)
    
calcXYZfrom3beam_bis()
#gc.collect()

Assuming ADCP is top mounted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def calcENUfromXYZ():
    def M_xyz2enu(heading,pitch,roll):
        hh = np.pi*(heading-90)/180
        pp = np.pi*pitch/180
        rr = np.pi*roll/180
        
        _H = np.array([
            [np.cos(hh),np.sin(hh),0], 
            [-np.sin(hh),np.cos(hh),0], 
            [0,0,1]
        ])
        _P = np.array([
            [np.cos(pp), -np.sin(pp)*np.sin(rr), -np.cos(rr)*np.sin(pp)] ,
            [0, np.cos(rr), -np.sin(rr)] , 
            [ np.sin(pp), np.sin(rr)*np.cos(pp), np.cos(pp)*np.cos(rr)]
        ])
        
        _R = _H@_P
        return _R

    H = ADCP['Heading'].values
    P = ADCP['Pitch'].values
    R = ADCP['Roll'].values

    E = ADCP['X'].values.copy()
    N = ADCP['Y'].values.copy()
    U = ADCP['Z'].values.copy()

    r,c = np.shape(E)

    for i in tqdm(range(r)):
        XYZ2ENU = M_xyz2enu(H[i],P[i],R[i])
        for j in range(c):
            E[i,j], N[i,j], U[i,j] = XYZ2ENU @ [E[i,j], N[i,j], U[i,j]]

    ADCP['E'] = (['time','bin'], E )
    ADCP['N'] = (['time','bin'], N )
    ADCP['U'] = (['time','bin'], U )

calcENUfromXYZ()
gc.collect()

100%|███████████████████████████████████████████████████████████████████████| 183468/183468 [00:15<00:00, 11613.73it/s]


46

In [10]:
plt.figure()

PD = ADCP['Pitch'] < 0
PU = ADCP['Pitch'] > 0

plt.subplot(511)
_ = plt.hist(ADCP.isel(bin=0)['X'][PD].values,np.linspace(-1,1,100),color='r')
_ = plt.hist(ADCP.isel(bin=0)['X'][PU].values,np.linspace(-1,1,100),color='b')
plt.title('Glider moving forward so expect X negative')

plt.subplot(513)
_ = plt.hist(ADCP.isel(bin=0)['U'][PD].values,np.linspace(-1,1,100),color='r')
plt.title('Glider diving so expect U positive')

plt.subplot(515)
_ = plt.hist(ADCP.isel(bin=0)['U'][PU].values,np.linspace(-1,1,100),color='b')
plt.title('Glider climbing so expect U negative')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Glider climbing so expect U negative')

# LADCP

In [ ]:
#Create array of dives number (dive == between two resurface)

ADCP_time = np.asarray(ADCP['time']/(10**9)) #ADCP time in second
NAV_time = np.asarray(sx.date2float(data.data['Timestamp'])) #NAV time in second

ADCP_depth = np.asarray(gsw.z_from_p(ADCP['Pressure'],lat=55))

ADCP_DiveNumber = np.zeros_like(ADCP_depth,dtype='int')
ADCP_DiveNumber[ADCP_depth >-5]=1
ADCP_DiveNumber = np.diff(ADCP_DiveNumber,prepend=1)
ADCP_DiveNumber[ADCP_DiveNumber==-1] = 1 #------------------------------ !!!!! ou bien = 0

ADCP_DiveNumber_time=ADCP_time[ADCP_DiveNumber==1] #Start time of each dive
ADCP_DiveNumber_time = np.insert(ADCP_DiveNumber_time, 0, ADCP_time[0])

ADCP_DiveNumber = np.cumsum(ADCP_DiveNumber)

#print(ADCP_DiveNumber[4900:5500])
print(len(ADCP_DiveNumber_time))
print(  (ADCP_DiveNumber[-1]+1)  )

In [ ]:
#Calculate shear profile for each ADCP sample (1 each 10 seconds)

print(len(ADCP['E']))
print(len(ADCP['E'][0]))
print(len(ADCP_DiveNumber))

ADCP['Shear_E'] = (['time','bin'], ADCP['E'].values.copy() )
ADCP['Shear_N'] = (['time','bin'], ADCP['N'].values.copy() )

for t in tqdm(range(len(ADCP['E']))):
    ADCP['Shear_E'][t]=np.gradient(ADCP['E'][t],ADCP['bin_depth'][t])
    ADCP['Shear_N'][t]=np.gradient(ADCP['N'][t],ADCP['bin_depth'][t])

In [ ]:
#--------------- BROUILLON ---------------
t_start = ADCP_DiveNumber_time[10]
t_end = ADCP_DiveNumber_time[10+1]
print(t_start)
print(t_end)
print([ (ADCP_time>t_start) & (ADCP_time<t_end) ])
Shear_samples_to_consider=ADCP_time[ (ADCP_time>t_start) & (ADCP_time<t_end) ]

print(Shear_samples_to_consider)

In [ ]:
#Construct a map of shear with respect to depth 

nDive = (ADCP_DiveNumber[-1]+1) #Number of dive for the deployment, first dive : nDive=0
dz=2 #[m]
H=400 #[m] take H larger than what the ADCP can reach during the dive
Nz=(np.floor(H/dz)).astype('int')

#Shear_East=np.zeros((nDive,Nz))
Shear_East=np.empty((nDive,Nz))
Shear_East[:] = np.nan


Shear_North=np.empty((nDive,Nz))
Shear_North[:] = np.nan

print(Shear_East[10,53]) #This is Shear for dive number 10 and depth 53*dz=53*2=106m

for d in range(ADCP_DiveNumber[-1]+1):
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1]
        
    #Shear samples to consider
    East_Shear_samples_to_consider=(ADCP['Shear_E'])[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    East_Shear_samples_to_consider=East_Shear_samples_to_consider.values.ravel()
    
    North_Shear_samples_to_consider=(ADCP['Shear_N'])[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    North_Shear_samples_to_consider=North_Shear_samples_to_consider.values.ravel()
    
    #Bin depth samples to consider
    Bin_depth_samples_to_consider=(ADCP['bin_depth'])[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    Bin_depth_samples_to_consider=Bin_depth_samples_to_consider.values.ravel()
    Bin_depth_samples_to_consider=(np.floor(-Bin_depth_samples_to_consider/dz)).astype('int')
    
    #Using dataframe to calculate the average shear for each depth dz
    df = pd.DataFrame({'nz': Bin_depth_samples_to_consider,
                   'Shear_E': East_Shear_samples_to_consider,
                   'Shear_N': North_Shear_samples_to_consider})
    df = df.groupby(['nz']).mean()
    S_E=(df['Shear_E'])[df.index>=0]
    S_N=(df['Shear_N'])[df.index>=0]
    (Shear_East[d])[0:len(S_E)]=S_E[0:len(S_E)]
    (Shear_North[d])[0:len(S_N)]=S_N[0:len(S_N)]
    


In [ ]:
%matplotlib widget
plt.close('all')
ndive_to_show=150

fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
X=Shear_East[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
plt.plot(X,Y,'.')
plt.plot(X,Y)
plt.grid()
plt.xlabel('East component of shear [m/s /m]')
plt.ylabel('Depth [m]')
plt.title('Dive number : ' + str(ndive_to_show))


ax2 = fig.add_subplot(1, 2, 2)
X=Shear_North[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
plt.plot(X,Y,'.')
plt.plot(X,Y)
plt.grid()
plt.xlabel('North component of shear [m/s /m]')
plt.ylabel('Depth [m]')
plt.title('Dive number : ' + str(ndive_to_show))

plt.tight_layout()
fig.canvas.draw_idle()
plt.show()

In [ ]:
#Compute barotropic speed :
Barotropic_East=np.empty((nDive,Nz))
Barotropic_North=np.empty((nDive,Nz))

for d_n in range(len(Barotropic_East)) :
    for n_z in range(len(Barotropic_East[0])) :
        Integral_East=np.trapz(  y=(Shear_East[d_n])[0:n_z]  , dx=-dz) #We integrate from the surface to the bottom, so negative 
        Barotropic_East[d_n,n_z]=Integral_East
        
        Integral_North=np.trapz(  y=(Shear_North[d_n])[0:n_z]  , dx=-dz) #We integrate from the surface to the bottom, so negative 
        Barotropic_North[d_n,n_z]=Integral_North
    
    Barotropic_East[d_n]=Barotropic_East[d_n]-np.nanmean(Barotropic_East[d_n])
    Barotropic_North[d_n]=Barotropic_North[d_n]-np.nanmean(Barotropic_North[d_n])

In [ ]:
from mpl_toolkits.mplot3d import axes3d
from matplotlib.widgets import Slider, Button

X_east=Barotropic_East[~np.isnan(Barotropic_East[:,10])]
X_north=Barotropic_North[~np.isnan(Barotropic_North[:,10])]


%matplotlib widget
plt.close('all')
ndive_to_show=150

fig = plt.figure()

ax = fig.add_subplot(1, 2, 1)
X1=X_east[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
dot_East, = plt.plot(X1,Y,'.')
line_East, = plt.plot(X1,Y)
plt.xlim([-0.7, 0.7])
plt.ylim([0, -380])
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('East baroclinic speed [m/s]')
plt.ylabel('Depth')

ax2 = fig.add_subplot(1, 2, 2)
X2=X_north[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
dot_North, = plt.plot(X2,Y,'.')
line_North, = plt.plot(X2,Y)
plt.xlim([-0.7, 0.7])
plt.ylim([0, -380])
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('North baroclinic speed [m/s]')
plt.ylabel('Depth')
plt.tight_layout()


plt.subplots_adjust(left=0.25, bottom=0.25)
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
nDive_slider = Slider(
    ax=axfreq,
    label='Dive number',
    valstep=1,
    valmin=0,
    valmax=len(X_north)-1,
    valinit=np.floor((len(X_north)-1)/2),
)


def update(val):
    line_East.set_xdata(  X_east[ nDive_slider.val]  )
    dot_East.set_xdata(  X_east[ nDive_slider.val]  )
    line_North.set_xdata(  X_north[ nDive_slider.val]  )
    dot_North.set_xdata(  X_north[ nDive_slider.val]  )
    fig.canvas.draw_idle()

nDive_slider.on_changed(update)









fig2 = plt.figure()
ax3 = fig2.gca(projection='3d')
zline = np.linspace(-1, -H-1, num=Nz)
ax3.set_xlim3d(-0.5, 0.5)
ax3.set_ylim3d(-0.5, 0.5)
ax3.set_zlim3d(0, -140/100)
ax3.set_xlabel('East speed [m/s]')
ax3.set_ylabel('North speed [m/s]')
ax3.set_zlabel('Depth [m]')
ax3.set_box_aspect((1, 1, 1))
arrow_length_ratio=0.1
ax3.plot3D(zline*0, zline*0, zline/100, 'gray')
ax3.quiver(zline*0, zline*0, zline/100, X1, X2, zline*0)
ax3.quiver([0.4], [-0.4], [-140/100], [0], [0.5], [0], color='r', pivot='tail', length=1, arrow_length_ratio=0.3)
plt.gca().invert_zaxis()
plt.tight_layout()



fig.canvas.draw_idle()
fig2.canvas.draw_idle()
plt.show()

In [ ]:
#--------------- BROUILLON ---------------

nDive = (ADCP_DiveNumber[-1]+1) #Number of dive for the deployment, first dive : nDive=0
dz=2 #[m]
H=400 #[m] take H larger than what the ADCP can reach during the dive
Nz=(np.floor(H/dz)).astype('int')
Shear_East=np.zeros((nDive,Nz))

A=(ADCP['bin_depth'])[10000:10900]
A=A.values.ravel()
#print(A)
A=(np.floor(-A/2)).astype('int')
#print(A)
B=(ADCP['Shear_E'][10000:10900])
B=B.values.ravel()
#print(B)


df = pd.DataFrame({'nz': A,
                   'Shear_E': B})

df = df.groupby(['nz']).mean()

C=(df['Shear_E'])[df.index>=0]
print(len(C))
print(C[0:len(C)])
print(C)

# Fast Fourier Transform of ADCP signal

In [ ]:
from scipy.fft import fft, fftfreq

#Compute average dive time for the entire flight ---------------------------------------------------------------
diveNum= np.asarray(  data.data['diveNum'].values  )
diveTime= np.asarray(  sx.date2float(data.data['Timestamp']).values  )
index=np.zeros( (diveNum[-1],) , dtype=int)
for i in range(diveNum[-1]) :
    index[i] = np.argmax(diveNum==i+1)
for i in range(diveNum[-1]) :
    index[i] = int(diveTime[index[i]])
T_dive=np.mean(np.diff(index))
print("Average dive period [s] : ")
print(T_dive)
f_dive=1/T_dive
print("Average dive frequency [Hz] : ")
print(f_dive)


#Compute average dive time for the interval considered ---------------------------------------------------------------

x=30#Use this variable to chose the interval

time=sx.date2float(data.data['Timestamp'])
t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

diveNum= np.asarray(  data.data['diveNum'].values  )[i1:i2]
diveTime= np.asarray(  sx.date2float(data.data['Timestamp']).values  )[i1:i2]
index=np.zeros( (diveNum[-1]-diveNum[0],) , dtype=int)
for i in range(diveNum[-1]-diveNum[0]) :
    index[i] = np.argmax(diveNum==i+diveNum[0])
for i in range(diveNum[-1]-diveNum[0]) :
    index[i] = int(diveTime[index[i]])
T_dive=np.mean(np.diff(index))
print("Average dive period [s] : ")
print(T_dive)
f_dive=1/T_dive
print("Average dive frequency [Hz] : ")
print(f_dive)
#----------------------------------------------------------------------------------------------------------------------



#Compute FFT -------------------------------------------------------------------------------------------------------------
ADCP_Time=ADCP.time.values.astype('float')/(10**9)
ADCP_spd_xyz = np.asarray(  np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)  )


#Interval to study
t = np.nanpercentile(ADCP_Time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(ADCP_Time - x1)).argmin()
i2=(np.abs(ADCP_Time - x2)).argmin()



#Raw data
N = len(ADCP_Time[i1:i2])
T = 10
x = ADCP_Time[i1:i2]
y = ADCP_spd_xyz[i1:i2]


#smooth data
#y=smooth(y,3)

#Delete the last data because absurd result in time
#N = len(ADCP_Time[0:-10])
#T = 10
#x = ADCP_Time[0:-10]
#y = ADCP_spd_xyz[0:-10]


#Delete the constant part of speed
print("Average ADCP speed [m/s] :")
print(np.mean(y))
y=y-np.mean(y)

#Sampled data
#T = 50
#x=x[::5]
#y=y[::5]
#N = len(x)


yf = fft(y)
xf = fftfreq(N, T)[:N//2]

#Plotting figure ------------------------------------------------------------------------
plt.close('all')
fig = plt.figure()


ax0 = fig.add_subplot(2, 1, 1)
T=pd.to_datetime(x, unit='s', utc=True, origin='unix', cache='False')
plt.plot(T, y)
plt.title('(ADCP speed) - (ADCP average speed)')
if (T[0].strftime('%d %B %Y')==T[-1].strftime('%d %B %Y')) :
    plt.xlabel(T[0].strftime('%d %B %Y'))
else :
    plt.xlabel(T[0].strftime('%d %B %Y') + ' - ' + T[-1].strftime('%d %B %Y'))
plt.ylabel('Speed [m/s]')
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.grid()

ax = fig.add_subplot(2, 1, 2)
plt.plot(xf, 2.0/N*np.abs(yf[0:N//2]), label="FFT(ADCP speed)")
plt.plot([f_dive,f_dive], [-1,1], label="Average dive frequency")
plt.xlabel('Frequency [Hz]')
plt.title('FFT(ADCP speed - ADCP average speed)')
ax.set_ylim([0,0.06])
ax.set_xscale('log')
#ax.set_xlim([0,0.01])
plt.grid()


plt.legend()
plt.tight_layout()
plt.show()

# Searching for errors

### Computation of useful variables for plotting (ex: spd error between Steady state VS ADCP)

In [11]:
#Useful functions
def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')
def fillGaps(x,y):
    f = interp1d(x[np.isfinite(x+y)],y[np.isfinite(x+y)], bounds_error=False, fill_value=np.NaN)
    return(f(x))



#Useful variables ----------------------------------------------------------------------------
time=sx.date2float(data.data['Timestamp'])

_u = np.remainder(data.data.profileNum,2) == 0
_d = np.remainder(data.data.profileNum,2) == 1

Temperature=fillGaps(time, data.data.Temperature)
pressure=fillGaps(time, data.data.pressure)
salinity=fillGaps(time, data.data.salinity)
Ballast=fillGaps(time, data.data.BallastPos/1000000)
lon=fillGaps(time, data.data.lon)
lat=fillGaps(time, data.data.lat)

Depth = gsw.z_from_p(pressure,lat) # m . Note depth (Z) is negative, so diving is negative dZdt      
dZdt = fillGaps(time, np.gradient(Depth,time))

SA = gsw.SA_from_SP(salinity, pressure, lon, lat)
CT = gsw.CT_from_t(SA, Temperature, pressure)
rho = gsw.rho(SA, CT, pressure)



# Interpolation of ADCP data in order to match with drone data (data.data) -----------------------------------------
smooth_coeff=50
ADCP_Time=ADCP.time.values.astype('float')/(10**9)

ADCP_spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)
ADCP_speed=np.interp(time, ADCP_Time, ADCP_spd_xyz)

ADCP_xz_spd=np.interp(time, ADCP_Time, np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Z']**2) )

ADCP_vert_spd = np.interp(time, ADCP_Time, ADCP.isel(bin=0)['U'])

ADCP_x_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['X'])
ADCP_y_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Y'])
ADCP_z_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Z'])
ADCP_pitch=np.deg2rad(np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Pitch']))
ADCP_hrz_spd =  (ADCP_x_spd)*np.cos(np.deg2rad(ADCP_pitch)) -(ADCP_z_spd)*np.sin(np.deg2rad(ADCP_pitch))

ADCP_pressure=np.interp(time, ADCP_Time, ADCP.Pressure.values.astype('float') )
ADCP_pitch=np.interp(time, ADCP_Time, ADCP.Pitch.values.astype('float') )
ADCP_roll=np.interp(time, ADCP_Time, ADCP.Roll.values.astype('float') )
ADCP_heading=np.interp(time, ADCP_Time, ADCP.Heading.values.astype('float') )


ADCP_depth = gsw.z_from_p(ADCP_pressure, lat)
ADCP_pressure_speed = np.gradient(ADCP_depth, time)

#Influence of drone rotation in ADCP measurement -------------------------------------------------------------------
omega = fillGaps(time, np.gradient(ADCP_pitch,time))

# STEADY STATE (1 CONSTRAINT)

In [12]:
importlib.reload(sx)
    
def flight_model():              
    flight = sx.SlocumModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               mass=60.772) # Verify mass number !!  
    flight.model_function()
    data.data['speed_vert_notRegressed'] = flight.speed_vert   
    flight.regress()     
    data.data['alpha'] = flight.alpha
    data.data['speed'] = flight.speed
    data.data['speed_vert'] = flight.speed_vert
    data.data['speed_horz'] = flight.speed_horz
    data.data['w_H2O'] = flight.w_H2O
    data.data['_valid'] = flight._valid     
    return flight
       
flight_old = flight_model()

C:\Users\johan\OneDrive\Bureau\PFE-Goteborg\Scripts\SXBQ.py:379: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * _dynamic_pressure / self.rho / self.area_w)


Initial parameters:  {'mass': 60.772, 'vol0': 0.059015, 'area_w': 0.24, 'Cd_0': 0.044178749994477656, 'Cd_1': 1.1050612498618673, 'Cl': 2.7177974996602754, 'comp_p': 4.5e-06, 'comp_t': 6.5e-05}
Non-optimised score: 0.18426958995293155
Regressing...
Optimization terminated successfully.
         Current function value: 0.012620
         Iterations: 470
         Function evaluations: 756
Optimised parameters:  {'mass': 60.772, 'vol0': 0.05945736976999342, 'area_w': 0.24, 'Cd_0': 0.0460799246893569, 'Cd_1': 1.0284917266869125, 'Cl': 2.5853912714505562, 'comp_p': 4.780543179098981e-06, 'comp_t': 9.272779281353907e-05}
Final Optimised score: 0.012619782807965222


In [13]:
aoa=fillGaps(time, data.data.alpha)
Pitch=fillGaps(time, np.deg2rad(data.data.Pitch)) #DO NOT USE !!! Data is wrong when mass is moving
Roll=fillGaps(time, np.deg2rad(data.data.Roll))

speed=data.data.speed.values.astype('float')
vertical_speed =fillGaps(time, data.data.speed_vert)
vertical_speed_notRegressed =fillGaps(time, data.data.speed_vert_notRegressed)
horizontal_speed = speed*np.cos(aoa+Pitch)

#Acceleration criteria validity --------------------------------------------------------------------------------------
L = 2.7 #length of glider
V = np.nanmean(data.data.speed) #Average speed of drone
a_max = V*V/(10*L)
a = np.gradient(speed,time)

validity = np.sign(np.abs(a)-a_max)

#Speed error SS1
spd_err=ADCP_speed-speed

In [14]:
%matplotlib widget
plt.close('all')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(data.data['Timestamp'].values.astype('float'), data.data.speed,'--k', alpha=0.5,label="steady flight model")
plt.plot(data.data['Timestamp'].values.astype('float'),ADCP_speed, alpha=0.7, color='g',label="ADCP")
plt.title("speed of glider")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])


def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([0,1.2])
step = np.diff(t)/100
update(30)


print((t[0] + 30*step)/10**9 )
print(t[0] + 30*step + step)
print(np.interp(1.59846038e+18, data.data['Timestamp'].values.astype('float'),  data.data.speed))
print(1.59847874e+18-1.59846038e+18)



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[1.59846038e+09]
[1.59847874e+18]
0.2631628133145244
18360000000000.0


#### Plotting influence of different parameters (Temperature, Pressure, Salinity, Pitch, Alpha, ...)

In [15]:
x=306.7

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

In [16]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


ax = fig.add_subplot(2, 1, 1)
X=data.data['Timestamp'][i1:i2]
Y1=dZdt[i1:i2]
Y2=data.data.speed_vert_notRegressed[i1:i2]
Y3=data.data.speed_vert[i1:i2]
#X=X[data.data.Depth[i1:i2]<-10]
#Y1=Y1[data.data.Depth[i1:i2]<-10]
#Y2=Y2[data.data.Depth[i1:i2]<-10]
#Y3=Y3[data.data.Depth[i1:i2]<-10]
plt.plot(X[10:-10], smooth(Y1,15)[10:-10], label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (not regressed)',linestyle='--',color='brown')
plt.plot(X,Y3,label='Steady state (regressed with R1)',linestyle='--')
plt.grid()
plt.title("Vertical glider speed")
plt.legend()
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("speed [m/s]")
#ax.set_ylim([-1,1])

ax2 = fig.add_subplot(2, 1, 2)
X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=speed[i1:i2]
#X=X[data.data.Depth[i1:i2]<-10]
#Y1=Y1[data.data.Depth[i1:i2]<-10]
#Y2=Y2[data.data.Depth[i1:i2]<-10]
plt.plot(X,Y1,label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (regressed R1)',linestyle='--')
plt.grid()
plt.title("Vertical glider speed")
plt.legend()
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("speed [m/s]")
ax2.set_ylim([0,1])


fig.canvas.draw_idle()
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
%matplotlib widget

#Steady_State_0_spd_err = dZdt[flight_old.depth<-5] - (data.data['speed_vert_notRegressed'].values.copy())[flight_old.depth<-5] #[data.data['_valid']]
Steady_State_1_spd_err = dZdt[flight_old.depth<-5] - flight_old.speed_vert[flight_old.depth<-5] #[data.data['_valid']]

mu=np.nanmean(Steady_State_1_spd_err)
sigma=np.nanstd(Steady_State_1_spd_err)

plt.close('all')
plt.figure()
plt.subplot(111)
#hist_ss0 = plt.hist(Steady_State_0_spd_err,np.linspace(-0.5,0.5,300), density = True, alpha=0.5, label ='Vertical speed error (no regression)')
hist_ss1 = plt.hist(Steady_State_1_spd_err,np.linspace(-0.5,0.5,500), density = True)
#plt.ylim([0,18])
ylim = plt.ylim()
plt.plot([],[],':k', label='67% and 95%')
for pt in np.nanpercentile(Steady_State_1_spd_err,[5,50-34.1,50,50+34.1,95]):
    plt.plot([pt,pt],ylim,':k')
#plt.plot([mu_0,mu_0],ylim,'-b',label='\u03BC ')
plt.plot([mu,mu],ylim,'-r',label='\u03BC')
plt.legend()
plt.xlabel("Speed error [m/s]")
plt.title('Normalized histogram of vertical speed error (Steady State R1) \n (\u03BC = '+str(round(mu,4))+' m/s ; \u03C3 = '+str(round(sigma,4)) +' m/s)'  )
plt.xlim([-0.2,0.2])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
#plt.scatter(data.data['Timestamp'].values.astype('float'),spd_err, 1,c=data.data.Temperature,cmap='coolwarm', alpha=1)
#plt.scatter(time[_u],spd_err[_u], 1,'b', alpha=1,label='upcast')
X=data.data['Timestamp'][i1:i2]
Y=spd_err[i1:i2]
C=a[i1:i2]
#X=X[Depth[i1:i2]<-10]
#Y=Y[Depth[i1:i2]<-10]
#C=C[Depth[i1:i2]<-10]


plt.scatter(X,Y, 1,c=C,cmap='coolwarm', alpha=1, label='Total speed error')
cbar = plt.colorbar()
cbar.set_label('acceleration [m2/s]')
plt.clim([-0.0005,0.0005])
plt.grid()
plt.title("speed error (for depth > 1m)")
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("speed error [m/s]")
plt.legend()
ax.set_ylim([-0.8,0.8])
fig.canvas.draw_idle()


xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
%matplotlib widget

Steady_State_1_spd_err = ADCP_speed[flight_old.depth<-5] - flight_old.speed[flight_old.depth<-5] #[data.data['_valid']]
mu=np.nanmean(Steady_State_1_spd_err)
sigma=np.nanstd(Steady_State_1_spd_err)

plt.close('all')
plt.figure()
plt.subplot(111)
hist_ss1 = plt.hist(Steady_State_1_spd_err,np.linspace(-0.5,0.5,300), density = True)
plt.ylim([0,18])
ylim = plt.ylim()
for pt in np.nanpercentile(Steady_State_1_spd_err,[5,50-34.1,50,50+34.1,95]):
    plt.plot([pt,pt],ylim,':k')
plt.plot([mu,mu],ylim,'-r',label='\u03BC ')
plt.legend()
plt.xlabel("Speed error [m/s]")
plt.title('Normalized histogram of total speed error (Steady State R1) \n (\u03BC = '+str(round(mu,4))+' m/s ; \u03C3 = '+str(round(sigma,4)) +' m/s)'  )
plt.xlim([-0.4,0.4])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sensor Comparison

#### Vertical Speed

In [ ]:




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
X=data.data['Timestamp'][i1:i2]
Y1=smooth(-ADCP_vert_spd,1)[i1:i2]
Y2 = smooth(dZdt,15)[i1:i2]
plt.plot(X, Y1, alpha=0.7, label='ADCP UP speed')
plt.plot(X, Y2, alpha=0.7, label='Pressure sensor UP speed')
plt.grid()
plt.legend()
plt.title("Vertical speed sensor comparison")
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("Vertical speed [m/s]")
fig.canvas.draw_idle()
ax.set_ylim([-0.4,0.6])




Steady_State_1_spd_err = smooth(dZdt,15)[flight_old.depth<-5] - -ADCP_vert_spd[flight_old.depth<-5] #[data.data['_valid']]
mu=np.nanmean(Steady_State_1_spd_err)
sigma=np.nanstd(Steady_State_1_spd_err)

plt.subplot(1,2,2)
hist_ss1 = plt.hist(Steady_State_1_spd_err,np.linspace(-0.5,0.5,500), density = True)
ylim = plt.ylim()
plt.plot([],[],':k', label='67% and 95%')
for pt in np.nanpercentile(Steady_State_1_spd_err,[5,50-34.1,50,50+34.1,95]):
    plt.plot([pt,pt],ylim,':k')
plt.plot([mu,mu],ylim,'-r',label='\u03BC')
plt.legend()
plt.xlabel("Speed error [m/s]")
plt.title('Normalized histogram of vertical speed difference \n (\u03BC = '+str(round(mu,4))+' m/s ; \u03C3 = '+str(round(sigma,4)) +' m/s)'  )
plt.xlim([-0.12,0.12])
plt.show()







xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)

plt.show()

#### Horizontal Speed

In [ ]:
#       TO DO !!! (insert bottom tracking data here)

# Steady State with 2nd Speed constraint

In [49]:
importlib.reload(ss2)

flight = ss2.SteadyState_2_Model(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               0.5, #Tau
               ADCP_speed,
               mass=60.772) # Verify mass number !!

C:\Users\johan\OneDrive\Bureau\PFE-Goteborg\Scripts\SteadyState2Constraints.py:286: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * _dynamic_pressure / self.rho / self.area_w)


In [50]:
flight.model_function()

In [51]:
flight.Apply_lowpass_filter()

In [52]:
flight.regress()

Initial parameters:  {'mass': 60.772, 'vol0': 0.05944908141441475, 'area_w': 0.24, 'Cd_0': 0.04675839293615225, 'Cd_1': 2.149734099962422, 'Cd': 0.05, 'Cl': 1.9419712004293288, 'alpha_stall': 10, 'alpha_linear': 10, 'comp_p': 4.723279317627415e-06, 'comp_t': 8.676627535570479e-05, 'SSStau': 18.5}
Non-optimised score: 0.02286379886365868
Regressing...
Optimization terminated successfully.
         Current function value: 0.022841
         Iterations: 198
         Function evaluations: 324
Optimised parameters:  {'mass': 60.772, 'vol0': 0.05944916250066814, 'area_w': 0.24, 'Cd_0': 0.0466570666788363, 'Cd_1': 2.2662401386649496, 'Cd': 0.05, 'Cl': 1.9430713107807096, 'alpha_stall': 10, 'alpha_linear': 10, 'comp_p': 4.700117312553605e-06, 'comp_t': 8.745887320310856e-05, 'SSStau': 18.5}
Final Optimised score: 0.022841242861073075


In [ ]:
print(np.nanmean(   np.abs(flight.alpha[flight._valid])   ))
print(np.nanmean(   np.abs(flight.pitch[flight._valid])   ))
print(np.nanmean(   np.abs(flight.alpha[flight._valid] + flight.pitch[flight._valid])   ))
print(np.nanmean(   np.abs((flight.F_B[flight._valid] -flight.F_g[flight._valid])*np.sin(flight.alpha[flight._valid] + flight.pitch[flight._valid]))   ))
print(np.nanmean(   np.abs((flight.F_B[flight._valid] -flight.F_g[flight._valid])*np.sin( flight.pitch[flight._valid]))   ))


print(  0.0465087374050051 + 1.27273918009592*(0.07260724645085814**2)  )
print(np.nanmean(   np.abs(flight.rho[flight._valid])   ))

TAU=(60.772+3.46)*np.sqrt(  2/(1027.397838710252*0.24*0.05321837938936768*2*0.9970374456177116)  )
print("tau = " + str(TAU) + "s")
print(np.nanmean(   90*np.abs(np.sin(flight.alpha[flight._valid] ))))

In [53]:
x=30 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()



#Plot vertical speed -----------------------------------------------------------------------------------------
ax = fig.add_subplot(2, 2, 1)

X=data.data['Timestamp'][i1:i2]
Y1=smooth(dZdt, 15)[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1,label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax2 = fig.add_subplot(2, 2, 2)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]
Y4=flight.speed_filtered[i1:i2]

plt.plot(X,Y1,label='ADCP')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')
#plt.plot(X,Y4,label='Semi dynamic',linestyle=':',color='black')

plt.legend()
plt.grid()
plt.title("Total glider speed")
plt.ylabel("speed [m/s]")
ax2.set_ylim([-0.1,1])





x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

#Plot vertical speed -----------------------------------------------------------------------------------------
ax3 = fig.add_subplot(2, 2, 3)

X=data.data['Timestamp'][i1:i2]
Y1=smooth(dZdt, 15)[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1,label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax3.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax4 = fig.add_subplot(2, 2, 4)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]
Y4=flight.speed_filtered[i1:i2]

plt.plot(X,Y1,label='ADCP')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')
#plt.plot(X,Y4,label='Semi dynamic',linestyle=':',color='black')

plt.legend()
plt.grid()
plt.title("Total glider speed")
plt.ylabel("speed [m/s]")
ax4.set_ylim([-0.1,1])




fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    fig.suptitle(X[0].strftime('%d %B %Y'))
else :
    fig.suptitle(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
    
#fig.suptitle('This is a somewhat long figure title', fontsize=16)

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
#plt.gcf().axes[2].xaxis.set_major_formatter(xformatter)
#plt.gcf().axes[3].xaxis.set_major_formatter(xformatter)
#plt.tight_layout()
plt.show()


print(i1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

563065


In [ ]:
x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()





#Plot vertical speed -----------------------------------------------------------------------------------------
ax = fig.add_subplot(2, 2, 1)

X=data.data['Timestamp'][i1:i2]
Y1=smooth(dZdt,15)[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1,label='Pressure sensor')
#plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax2 = fig.add_subplot(2, 2, 2)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]

plt.plot(X,Y1,label='ADCP')
#plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Total glider speed")
ax2.set_ylim([-0.4,0.8])




#Plot vertical speed  error -----------------------------------------------------------------------------------------
ax3 = fig.add_subplot(2, 2, 3)

X=data.data['Timestamp'][i1:i2]
Y=smooth(dZdt,15)[i1:i2]-flight.speed_vert[i1:i2]
C=np.abs(   np.gradient(flight.speed_vert[i1:i2],flight.time[i1:i2])   )

plt.scatter(X,Y,3,c=C,cmap='plasma_r', alpha=1)
plt.clim([0,0.04])

cbar = plt.colorbar(orientation="horizontal", pad=0.2)
cbar.ax.set_title('|az| [m2/s]', rotation=0)

plt.grid()
plt.title("Vertical speed error")
plt.ylabel("speed error [m/s]")
ax3.set_ylim([-0.3,0.42])



#Plot total speed  error -----------------------------------------------------------------------------------------
ax4 = fig.add_subplot(2, 2, 4)

X=data.data['Timestamp'][i1:i2]
Y=ADCP_speed[i1:i2]-flight.speed[i1:i2]
C=np.abs(   np.gradient(flight.speed[i1:i2],flight.time[i1:i2])   )

plt.scatter(X,Y,3,c=C,cmap='plasma_r', alpha=1)
plt.clim([0,0.08])

cbar = plt.colorbar(orientation="horizontal", pad=0.2)
cbar.ax.set_title('|a| [m2/s]', rotation=0)

plt.grid()
plt.title("Total speed error")
ax4.set_ylim([-0.3,0.42])




fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    fig.suptitle(X[0].strftime('%d %B %Y'))
else :
    fig.suptitle(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
    
#fig.suptitle('This is a somewhat long figure title', fontsize=16)

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[2].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[4].xaxis.set_major_formatter(xformatter)  # /!\ les cbar sont considérées comme des axes x
#plt.tight_layout()
plt.show()

In [ ]:
%matplotlib widget

Steady_State_2_spd_err = ADCP_speed[flight.depth<-5] - flight.speed[flight.depth<-5] #[flight._valid] or [flight.depth<-5]
mu=np.nanmean(Steady_State_2_spd_err)
sigma=np.nanstd(Steady_State_2_spd_err)

plt.close('all')
plt.figure()
plt.subplot(111)
hist_ss1 = plt.hist(Steady_State_2_spd_err,np.linspace(-0.5,0.5,300), density = True)
plt.ylim([0,18])
ylim = plt.ylim()
for pt in np.nanpercentile(Steady_State_2_spd_err,[5,50-34.1,50,50+34.1,95]):
    plt.plot([pt,pt],ylim,':k')
plt.plot([mu,mu],ylim,'-r',label='\u03BC')
plt.legend()
plt.xlabel("Speed error [m/s]")
plt.title('Normalized histogram of total speed error (Steady State R2) \n (\u03BC = '+str(round(mu,4))+' m/s ; \u03C3 = '+str(round(sigma,4)) +' m/s)'  )
plt.xlim([-0.4,0.4])
plt.show()

print(np.nanstd(Steady_State_2_spd_err))
print(np.nanmean(Steady_State_2_spd_err))

In [ ]:
%matplotlib widget

Steady_State_2_filtered_spd_err = ADCP_speed[flight.depth<-5] - flight.speed_filtered[flight.depth<-5] #[flight._valid]
mu=np.nanmean(Steady_State_2_filtered_spd_err)
sigma=np.nanstd(Steady_State_2_filtered_spd_err)

plt.close('all')
plt.figure()
plt.subplot(111)
hist_ss1 = plt.hist(Steady_State_2_filtered_spd_err,np.linspace(-0.5,0.5,300), density = True)
plt.ylim([0,18])
ylim = plt.ylim()
for pt in np.nanpercentile(Steady_State_2_filtered_spd_err,[5,50-34.1,50,50+34.1,95]):
    plt.plot([pt,pt],ylim,':k')
plt.plot([mu,mu],ylim,'-r',label='\u03BC')
plt.legend()
plt.xlabel("Speed error [m/s]")
plt.title('Normalized histogram of total speed error (Semi Dynamic R2) \n (\u03BC = '+str(round(mu,4))+' m/s ; \u03C3 = '+str(round(sigma,4)) +' m/s)'  )
plt.xlim([-0.4,0.4])
plt.show()

print(np.nanstd(Steady_State_2_filtered_spd_err))
print(np.nanmean(Steady_State_2_filtered_spd_err))

In [ ]:
x=30


t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()




#Plot total speed -----------------------------------------------------------------------------------------
ax2 = fig.add_subplot(2, 1, 1)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]

plt.plot(X,Y1,label='ADCP')
#plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')
plt.ylabel("speed [m/s]")

plt.legend()
plt.grid()
plt.title("Total glider speed")
ax2.set_ylim([0,0.8])







#-----------------------------------------------------------------------------------------
ax = fig.add_subplot(2, 1, 2)

X=data.data['Timestamp'][i1:i2]
Y=np.gradient(flight.speed[i1:i2],flight.time[i1:i2])

V=np.nanmean(flight.speed)
L=2.0
a_max = V*V/(10*L)

plt.plot(X,Y, color='red', linestyle='dashed', label='Steady state acceleration')
plt.plot([X[0],X[-1]],[a_max,a_max], '--k')
plt.plot([X[0],X[-1]],[-a_max,-a_max], '--k', label='Validity domain')

plt.legend()
plt.grid()
plt.ylabel("acceleration [m2/s]")
plt.title("Acceleration criteria")
ax.set_ylim([-0.08,0.11])

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
#plt.tight_layout()
plt.show()

# Dynamic flight model -------------------------------------------------------------------------

In [ ]:
import dynamic_model as DM
importlib.reload(DM)

flight_dyn = DM.DynamicFlightModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               ADCP_speed)


flight_dyn.set_initial_conditions(flight.speed_horz[558109],flight.speed_vert[558109])
flight_dyn.solveRK4()

In [ ]:
flight_dyn.regress()

#### Comparison of  TOTAL SPEED (Dyanmic VS Static VS ADCP)

In [ ]:
x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



X=data.data['Timestamp'][i1:i2]


ADCP_alpha=np.arctan2(-ADCP_vert_spd,-ADCP_hrz_spd)-ADCP_pitch



#Dynamic results

Dyn_Speed= np.sqrt(flight_dyn.w**2+flight_dyn.u**2)
Dyn_Speed=np.interp(time, flight_dyn.t, Dyn_Speed)
Dyn_Speed=Dyn_Speed[i1:i2]

Dyn_Speed_vert=np.interp(time, flight_dyn.t, flight_dyn.w)
Dyn_Speed_vert=Dyn_Speed_vert[i1:i2]

Dyn_alpha=np.arctan2(flight_dyn.w,flight_dyn.u)-np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch)
Dyn_alpha=np.interp(time, flight_dyn.t, Dyn_alpha)
Dyn_alpha=Dyn_alpha[i1:i2]

#Fb, Fg = flight_dyn.compute_FB_and_Fg(g, rho, pressure, ballast, temperature)
#for i in range(len(flight_dyn.t)) :
#    L, D=flight_dyn.compute_lift_and_drag(pitch[i], rho[i], flight_dyn.u[i], flight_dyn.w[i])
#L, D=flight_dyn.compute_lift_and_drag(pitch, rho, flight_dyn.u, flight_dyn.w)
#Fx=np.sin(pitch + alpha)*L-np.cos(pitch + alpha)*D
#Fy=Fb - Fg -np.cos(pitch + alpha)*L -np.sin(pitch + alpha)*D





#Plotting figures
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(2, 1, 1)

#ADCP speed
#plt.plot(time, -ADCP_vert_spd, 'b',label="w ADCP [m/s]") #ADCP speed interpolated 
#plt.plot(time, ADCP_x_spd, 'b',label="w ADCP [m/s]") #ADCP speed interpolated 
#plt.plot(time,-ADCP_hrz_spd,'b', alpha=1, label='ADCP speed') #ADCP speed not interpolated 
plt.plot(X,ADCP_speed[i1:i2], alpha=1, label='ADCP') #ADCP speed not interpolated 

#Pressure sensor
#plt.plot(X, dZdt[i1:i2], 'm',label="w Pressure sensor [m/s]")

#Dynamic model forces
#plt.plot(flight_dyn.t, ballast*1000, 'k')
#plt.plot(flight_dyn.t, pitch/1.9, '--k')
#plt.plot(flight_dyn.t, D/10, 'y')
#plt.plot(flight_dyn.t, L/10, 'y')

#Dynamic model pitch, depth, alpha, ...
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'g')
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'g')
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.depth), 'k')
#plt.plot(flight_dyn.t, (np.arctan2(flight_dyn.w,flight_dyn.u)-np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch)), '--r')


#Steady state speed
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), data.data.speed_horz),'--y',label="u stat [m/s]")
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), flight.speed_vert),'--y',label="w steady state [m/s]")
#plt.plot(X, flight.speed[i1:i2], '--',label="Steady state (R2)")


#Dynamic model Speed
#plt.plot(flight_dyn.t, flight_dyn.u, 'r', label="u dynamic [m/s]")
#plt.plot(flight_dyn.t, flight_dyn.w, 'r', label="w dynamic [m/s]")
#plt.plot(flight_dyn.t, np.sqrt(flight_dyn.w**2+flight_dyn.u**2), color='red', label="U dynamic model")
plt.plot(X, Dyn_Speed, color='red', label="Dynamic model")


#SEMI Steady state speed
plt.plot(X, flight.speed_filtered[i1:i2], ':k',label="Semi steady state (R2)")

plt.grid()
plt.ylabel("Speed [m/s]")
plt.title("Total speed")
plt.legend()
ax.set_ylim([0,0.8])






ax2 = fig.add_subplot(2, 1, 2)

plt.plot(X, smooth(dZdt, 15)[i1:i2],label="Pressure sensor")
#plt.plot(X, flight.speed_vert[i1:i2], '--',label="Steady state (R2)")
plt.plot(X, Dyn_Speed_vert, color='red', label="Dynamic model")
plt.plot(X, flight.speed_vert_filtered[i1:i2], linestyle=':',color='k',alpha=0.6,label="Semi steady state (R2)")

#plt.plot(X, flight.alpha[i1:i2], 'm',label="Semi steady state (R2)")
#plt.plot(X, flight.vert_dir[i1:i2], 'y',label="Semi steady state (R2)")
#plt.plot(X, flight.ballast[i1:i2]*1000, 'b',label="Ballast")
#plt.plot(X, flight.pitch[i1:i2], 'y',label="Semi steady state (R2)")
#plt.plot(X, Dyn_alpha, 'y',label="Semi steady state (R2)")
#plt.plot(X, Pitch[i1:i2], 'y',label="Pitch NAV")
#plt.plot(X, ADCP_pitch[i1:i2], ':r',label="Pitch ADCP")

plt.grid()
plt.ylabel("Speed [m/s]")
plt.title("Vertical speed ")
plt.legend()
ax.set_ylim([0,0.8])

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))


print(X.values.astype('float')[0])

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.tight_layout()
plt.show()

#### Comparison of speed error (relative to ADCP) between dynamic and static

In [ ]:
x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



# Definition of error function
Dyn_Speed= np.sqrt(flight_dyn.w**2+flight_dyn.u**2)

Dyn_spd_err = ADCP_speed - np.interp(time, flight_dyn.t, Dyn_Speed)
Dyn_spd_vert_err = smooth(dZdt,15) - np.interp(time, flight_dyn.t, flight_dyn.w)
Dyn_spd_err=Dyn_spd_err[i1:i2]
Dyn_spd_vert_err=Dyn_spd_vert_err[i1:i2]

Stead_spd_err =  (ADCP_speed-flight.speed)[i1:i2]
Stead_spd_vert_err = (smooth(dZdt,15)-flight.speed_vert)[i1:i2]
X=data.data['Timestamp'][i1:i2]



#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


ax = fig.add_subplot(2, 1, 1)

plt.plot(X, Dyn_spd_err, 'r', alpha=1, label ='Dynamic model')
plt.plot(X, Stead_spd_err, '--',color='orange', alpha=1, label ='Steady state model')

plt.grid()
plt.legend()
plt.title("Total speed error")
plt.ylabel("speed error [m/s]")
ax.set_ylim([-0.3,0.3])



ax2 = fig.add_subplot(2, 1, 2)

plt.plot(X, Dyn_spd_vert_err, 'r', alpha=1, label ='Dynamic model')
plt.plot(X, Stead_spd_vert_err, '--',color='orange', alpha=1, label ='Steady state model')

plt.grid()
plt.legend()
plt.title("Vertical speed error")
plt.ylabel("speed error [m/s]")
ax2.set_ylim([-0.3,0.5])



fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.tight_layout()
plt.show()

#### Comparison of  HORIZONTAL SPEED (Dyanmic VS Static VS ADCP)

In [ ]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(time,horizontal_speed,'--y', alpha=0.6, label='Steady state Horizontal speed')
plt.plot(flight_dyn.t, flight_dyn.u, 'r',alpha=0.6, label="Dynamic Horizontal speed")
plt.scatter(time,smooth(-ADCP_hrz_spd,smooth_coeff), 1,'b', alpha=1, label='ADCP Horizontal speed') #Because : Vadcp = - Vdrone
plt.grid()
plt.legend()
plt.title("Horizontal speed")
plt.xlabel("t [s]")
plt.ylabel("speed error [m/s]")

t = np.nanpercentile(time,[0,100])
def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()
ax.set_ylim([-0.1,0.7])
step = np.diff(t)/100
update(30)

plt.show()